## Exercise 05.1 (random numbers)

- Using the `randint` function from the `random` module (https://docs.python.org/3/library/random.html#random.randint) to
  develop a function `dice_roll` that emulates the roll of a dice with $n$ sides. The number of sides `n` should an argument to the function.

- For $n=6$, devise and implement a test to check that it is a fair dice.

#### (a) Dice roll code:

In [3]:
import random

def dice_roll(n):
    # randint(a, b) returns a random integer b/w a and b both included
    return random.randint(1, n)

In [4]:
## tests ##
for n in range(1, 20):
    for j in range(100):
        value = dice_roll(n) 
        assert value >= 1 and value <= n

#### (b) Test for fairness

In [6]:
# To check for fairness, we roll the dice a large number of times and check how many times we get each value. 
# We expect to get each value close to $1/6$ of the times. 

dice_count = [0]*6
for i in range(1_000_000):
    roll = dice_roll(6)
    dice_count[roll - 1] += 1

for i in range(6):
    count = dice_count[i]
    avg = count / 1_000_000
    print(f"Average frequency of roll {i + 1}: {avg}")

## in the output frequency of each roll comes nearly 0.1667

Average frequency of roll 1: 0.167472
Average frequency of roll 2: 0.165958
Average frequency of roll 3: 0.166141
Average frequency of roll 4: 0.167072
Average frequency of roll 5: 0.166729
Average frequency of roll 6: 0.166628


## Exercise 05.2 (variance estimation)

For a random variable $X$, the variance of $X$ is defined as  

$$
\begin{align}
\mathrm{Var}\left[ X \right] &= \mathrm{E}\left[\left( X - \mu \right)^2\right]  \\ 
&= \mathrm{E}\left[ X^{2} \right] - \mathrm{E}\left[ X \right]^2
\end{align}
$$

where $\mathrm{E}$ is the 'expectation' (mean of something) and $\mu = \mathrm{E}(X)$ is the mean of $X$. If we have all data (the entire 'population'),
the variance can be computed from:

$$
\mathrm{Var}\left[ X \right] = \frac{\sum_{i=0}^{N-1} x^{2}_{i}}{N} - \left( \frac{ \left( \sum_{i=0}^{N-1} x_{i} \right)}{N} \right)^{2}
$$

Often, we only have a sample of data. For example, we might want to estimate the variance in height for students at a university using just a random sample of students. 
When using a sample from a larger data set to estimate the variance, the above formula has a *bias*. Therefore, it is common to use the *unbiased* estimator

$$
s^{2} = \left( \frac{\sum_{i=0}^{n-1} x^{2}_{i}}{n} - \left( \frac{ \left( \sum_{i=0}^{n-1} x_{i} \right)}{n} \right)^{2} \right) \frac{n}{n-1}
$$

to estimate the variance. In this exercise we will use the unbiased estimator.

1. Create a function that returns the estimated variance for a list of numbers based on the above equation. Test your function using 1 million samples drawn from a Gaussian distribution with a mean of 10 and a standard deviation of 3. Use your crsid to seed the random number generator
   
   *See hint below on how to create the sample.*

2. For a sample drawn from a distribution with mean $5 \times 10^6$ and standard deviation $2.0$, estimate the variance using (i) your function for estimating the variance, and (ii) using the `variance` function from the Python `statistics` module. Comment on and explain any significant differences in the results from (i) and (ii).

### Hint: sampling from a distribution

The function `random.gauss` can be used to sample a Gaussian distribution with a specified mean and standard deviation (square root of the variance) *N* times, e.g.:

In [7]:
import random

random.seed("gnw20")  # See the random number generator

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
x = [random.gauss(mu, sigma) for i in range(8)]
print(x)

[11.176693418313244, 12.08792069683015, 12.649052763696117, 9.845742673456446, 7.317196693264903, 6.876099163343957, 15.181269044169637, 12.844967463974331]


### Solution

1. Estimate variance

In [9]:
import random
random.seed("gnw20")

def estimate_unbiased_variance(data):
    n = len(data)
    mean = sum(data) / n
    s = 0
    for x in data:
        s += (x - mean)**2
    variance = s / (n - 1)  # unbiased
    return variance

In [12]:
## test with gaussian samples
random.seed("gnw20")

mu = 10.0
sigma = 3.0
samples = []
for _ in range(1_000_000):
    samples.append(random.gauss(mu, sigma))

estimated_var = estimate_unbiased_variance(samples)
print(f"Estimated variance: {estimated_var:.5f}")
print(f"Expected variance (sigma^2): {sigma**2:.5f}")

Estimated variance: 9.00211
Expected variance (sigma^2): 9.00000


2. Use the `statistics` module to estimate the variance, and compare the estimated variance using `statistics` to the estimated variance using your implementation for the variance estimation.

In [14]:
import statistics

# Large numbers example
random.seed(42)
samples_large = []
for _ in range(1_000_000):
    samples_large.append(random.gauss(5e6, 2.0))

var_custom = estimate_variance(samples_large)
var_stats = statistics.variance(samples_large)

print(f"Custom variance estimator: {var_custom}")
print(f"statistics.variance: {var_stats}")

Custom variance estimator: 3.997610294530491
statistics.variance: 3.9976102945305834


Notes: The difference between custom unbiased variance (using n-1) and statistics variance (using n) is negligible for large samples size as n approaches inf, the ratio n/n-1 approaches 1.

## Exercise 05.03 (optional, parallel processing)

Almost all modern computer processing units have multiple processing *cores*. To utilise the full performance of a processor, operations need to be performed in *parallel*, i.e. each processing core is given a task to perform.

Parallel computing is a very rich and technical area. To help exploit multi-core systems there are libraries that support parallel processing. Below is a simple example using the Python `multiprocessing` library.

In [ ]:
import multiprocessing

# On some operaring systems, you may need to uncomment the below line
# multiprocessing.set_start_method('fork')

def f(task):
    """A function that print the input argument and the id for the process that executes the function"""
    print(f"Task index {task}, process id: {multiprocessing.current_process()}\n")
     
    return str(multiprocessing.current_process())
        
        
# Using 3 'processes', execute the function 'f' four times (each time with a different argument)  
with multiprocessing.Pool(processes=3) as p:
    procs = p.map(f, [0, 1, 2, 3])  # Call function mysort three times
    
    print("Returned data (a list)") 
    print(procs)

In [1]:
import multiprocessing

# On some operaring systems, you may need to uncomment the below line
multiprocessing.set_start_method('fork')

def f(task):
    """A function that print the input argument and the id for the process that executes the function"""
    print(f"Task index {task}, process id: {multiprocessing.current_process()}\n")
     
    return str(multiprocessing.current_process())
        
        
# Using 3 'processes', execute the function 'f' four times (each time with a different argument)  
with multiprocessing.Pool(processes=3) as p:
    procs = p.map(f, [0, 1, 2, 3])  # Call function mysort three times
    
    print("Returned data (a list)") 
    print(procs)

Task index 0, process id: <ForkProcess name='ForkPoolWorker-1' parent=54427 started daemon>
Task index 1, process id: <ForkProcess name='ForkPoolWorker-2' parent=54427 started daemon>
Task index 2, process id: <ForkProcess name='ForkPoolWorker-3' parent=54427 started daemon>



Task index 3, process id: <ForkProcess name='ForkPoolWorker-1' parent=54427 started daemon>

Returned data (a list)
["<ForkProcess name='ForkPoolWorker-1' parent=54427 started daemon>", "<ForkProcess name='ForkPoolWorker-2' parent=54427 started daemon>", "<ForkProcess name='ForkPoolWorker-3' parent=54427 started daemon>", "<ForkProcess name='ForkPoolWorker-1' parent=54427 started daemon>"]


Consider the below code that creates a list of lists of sorted numbers.

In [4]:
def mysort(N):
    """Create a randomly ordered list of integers of length N, and return the sorted list"""
    # Create randomly ordered list
    x = random.sample(range(0, N), N)
 
    # Return sorted list of numbers
    return sorted(x)

def sorted_lists(N, p):
    """Create a list of sorted lists"""
    data = []
    for i in range(p):
        data.append(mysort(N))

%time x = sorted_lists(1000000, 5)

CPU times: user 1.5 s, sys: 31.6 ms, total: 1.53 s
Wall time: 1.53 s


Use the `multiprocessing` module to perform the above operation in parallel. Investigate how the processing time changes with the number of processes, and in particular the average time per 'create and sort operation' when changing the number of processes.

In [12]:
import multiprocessing
import time

def parallel_sorted_lists(N, p, num_processes):
    with multiprocessing.Pool(processes=num_processes) as pool:
        # purpose of passing [N]*p is to run mysort p times
        # everytime with N as argument
        result = pool.map(mysort, [N]*p)
    return result

for processes in [1, 2, 3, 4]:
    %time x = parallel_sorted_lists(1000000, 5, processes)
    avg_time_per_create_sort = x["total"] / 5
    print(avg_time_per_create_sort)
    # print(f"{processes} processes -> total time: {elapsed:.2f}s, avg per sort: {avg_time_per_task:.2f}s")

CPU times: user 260 ms, sys: 144 ms, total: 404 ms
Wall time: 1.97 s
CPU times: user 67.7 ms, sys: 40.9 ms, total: 109 ms
Wall time: 1.1 s
CPU times: user 67 ms, sys: 37.2 ms, total: 104 ms
Wall time: 854 ms
CPU times: user 67.6 ms, sys: 45.7 ms, total: 113 ms
Wall time: 809 ms
